# Parabolic DGA Demo (Type A₅, Non-3-Equal)

This notebook demonstrates the Sage module `parabolic_dga.sage`:

- builds the parabolic relative chain complex,
- computes Betti numbers over GF(2) and ℤ,
- verifies `d^2 = 0` and the Leibniz rule,
- reproduces Baryshnikov's case (A₅ non-3-equal): Betti = (1, 111, 20) over GF(2).

**Reference:** J. Cantarero – J. L. León-Medina, *The Cohomology Ring of Real Parabolic Arrangements*.

In [ ]:
# If you run this notebook from the repo root, use:
# load('parabolic_dga.sage')

# If the notebook is inside notebooks/, use:
load('../parabolic_dga.sage')

## Build Type A₅ and the non-3-equal arrangement

We remove all parabolic cosets whose index set J contains a consecutive block of length k−1 = 2.

In [ ]:
W, P, Plist = build_W_P("A", 5)
Delta = ideal_non_k_equal_A(W, Plist, k=3)  # non-3-equal
D = ParabolicDGA(W, P, Plist, Delta)

# Inspect chain group sizes by degree k = |J|
sizes = {k: len(v) for k, v in sorted(D.by_k.items())}
sizes

## Internal consistency checks

- Verify `d^2 = 0` over ℤ and GF(2)  
- Verify the Leibniz rule  
- (Optional) Sanity check Betti numbers for A₅ non-3-equal

In [ ]:
# Quick internal checks (no Betti check)
D.self_test(trials=30, check_betti=False, verbose=True)

In [ ]:
# Optional: also verify the expected Betti numbers for A5 non-3-equal
D.self_test(trials=30, check_betti=True, verbose=True)

## Betti numbers

- Over GF(2) we expect: (1, 111, 20) for degrees k = 0,1,2.  
- Over ℤ we compute **ranks** (torsion ignored).

In [ ]:
b2 = D.betti_numbers(base_ring=GF(2))
bZ = D.betti_numbers(base_ring=ZZ)
print("Betti over GF(2):", b2)
print("Betti over ZZ (ranks):", bZ)
((b2.get(0,0), b2.get(1,0), b2.get(2,0)) == (1, 111, 20))

## Minimal intersection product

We multiply two degree-1 basis cochains supported on `(e,(1,))` and `(e,(3,))` to get a degree-2 element.

In [ ]:
C1 = D.by_k.get(1, [])
idx1 = {c:i for i,c in enumerate(C1)}
e = W.one()

v1 = [0]*len(C1); v2 = [0]*len(C1)
if (e,(1,)) in idx1 and (e,(3,)) in idx1:
    v1[idx1[(e,(1,))]] = 1
    v2[idx1[(e,(3,))]] = 1

out_ZZ  = D.product_sparse(v1, 1, v2, 1, base_ring=ZZ)
out_GF2 = D.product_sparse(v1, 1, v2, 1, base_ring=GF(2))
out_ZZ, out_GF2

### Optional: Check if the product represents a nonzero cohomology class

We test whether the product is in `Ker(d^2)` but not in `Im(d^1)` over GF(2),
which indicates a nonzero class in `H^2`.

In [ ]:
mats2 = D.boundary_matrices(base_ring=GF(2))
d1 = mats2.get(1, None)
d2 = mats2.get(2, None)
C2 = D.by_k.get(2, [])
res = None
if d1 is not None and d2 is not None and len(C2)>0:
    from sage.all import vector as svec
    out2 = svec(GF(2), [int(a)%2 for a in out_ZZ])
    Im1  = d1.transpose().row_space()
    Ker2 = d2.transpose().right_kernel()
    res = (out2 in Ker2, out2 in Im1)
res